In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Reading dataset into csv

In [ ]:
import os
import csv

data_directory = '/content/drive/MyDrive/Capstone/op_spam_v1.4 3'


# Define a function to extract information from the filename
def parse_filename(filename):
    parts = filename.split('_')
    class_label = parts[0]
    hotel_name = parts[1]
    return class_label, hotel_name

def parse_foldername(name):
    parts = name.split('/')
    polarity, source = "", ""
    for part in parts:
        if part.split('_')[-1] == "polarity":
            polarity =  part.split('_')[0]
        elif len(part.split('_')) == 3 and part.split('_')[1] == "from":
            source = part.split('_')[2]

    return polarity, source



def recurse(data_directory, writer):

    for file in os.listdir(data_directory):
        if os.path.isfile(os.path.join(data_directory, file)) and file.endswith(".txt"):
            # print("we have got the end")
                # Traverse through files in the directory
            for filename in os.listdir(data_directory):
                if filename.endswith('.txt'):
                    file_path = os.path.join(data_directory, filename)
                    with open(file_path, 'r', encoding='utf-8') as file:
                        review_text = file.read()
                    class_label, hotel_name = parse_filename(filename)
                    polarity, source = parse_foldername(file_path)
                    writer.writerow([polarity,class_label, hotel_name, source, review_text])
            return
        elif os.path.isdir(os.path.join(data_directory, file)):
            #print("Inside folder ", os.path.join(data_directory, file))

            recurse(os.path.join(data_directory, file), writer)


# Define the header for the CSV file
header = ['Polarity', 'Deceptive', 'Hotel', 'Source', 'Review']

# Open the CSV file for writing
with open(data_directory+'/deceptive_opinion_spam.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)  # Write the header row
    recurse(data_directory, writer)

# Data Engineering


In [ ]:
import pandas as pd
import numpy as np
csvpath = "/content/drive/MyDrive/Capstone/op_spam_v1.4 3/deceptive_opinion_spam.csv"
data = pd.read_csv(csvpath, header=0, sep=',', quotechar='"')
data

,Polarity,Deceptive,Hotel,Source,Review
0,negative,d,hyatt,MTurk,The Hyatt Regency Chicago basically caters to ...
1,negative,d,fairmont,MTurk,My experience as Fairmont Chicago Millennium P...
2,negative,d,omni,MTurk,When my boyfriend and I checked into our room ...
3,negative,d,hyatt,MTurk,My wife and I stayed at the Hyatt Regency in C...
4,negative,d,omni,MTurk,I was extremely disappointed with my stay at t...
...,...,...,...,...,...
1595,positive,d,sheraton,MTurk,What a gorgeous hotel! From the moment we step...
1596,positive,d,homewood,MTurk,If you want the downtown experience of a lifet...
1597,positive,d,knickerbocker,MTurk,I stayed at the Millennium Knickerbocker Hotel...
1598,positive,d,homewood,MTurk,Downtown Chicago has a multitude of available ...


In [ ]:
data['Polarity'] = np.where(data['Polarity']=='positive', 1, 0)
data['Deceptive'] = np.where(data['Deceptive']=='t', 1, 0)
data

,Polarity,Deceptive,Hotel,Source,Review
0,0,0,hyatt,MTurk,The Hyatt Regency Chicago basically caters to ...
1,0,0,fairmont,MTurk,My experience as Fairmont Chicago Millennium P...
2,0,0,omni,MTurk,When my boyfriend and I checked into our room ...
3,0,0,hyatt,MTurk,My wife and I stayed at the Hyatt Regency in C...
4,0,0,omni,MTurk,I was extremely disappointed with my stay at t...
...,...,...,...,...,...
1595,1,0,sheraton,MTurk,What a gorgeous hotel! From the moment we step...
1596,1,0,homewood,MTurk,If you want the downtown experience of a lifet...
1597,1,0,knickerbocker,MTurk,I stayed at the Millennium Knickerbocker Hotel...
1598,1,0,homewood,MTurk,Downtown Chicago has a multitude of available ...


In [ ]:
def create_class(c):
    if c['Polarity'] == 1 and c['Deceptive'] == 1:
        return [1,1]
    elif c['Polarity'] == 1 and c['Deceptive'] == 0:
        return [1,0]
    elif c['Polarity'] == 0 and c['Deceptive'] == 1:
        return [0,1]
    else:
        return [0,0]

def specific_class(c):
    if c['Polarity'] == 1 and c['Deceptive'] == 1:
        return "TRUE_POSITIVE"
    elif c['Polarity'] == 1 and c['Deceptive'] == 0:
        return "FALSE_POSITIVE"
    elif c['Polarity'] == 0 and c['Deceptive'] == 1:
        return "TRUE_NEGATIVE"
    else:
        return "FALSE_NEGATIVE"

data['final_class'] = data.apply(create_class, axis=1)
data['given_class'] = data.apply(specific_class, axis=1)
data

,Polarity,Deceptive,Hotel,Source,Review,final_class,given_class
0,0,0,hyatt,MTurk,The Hyatt Regency Chicago basically caters to ...,"[0, 0]",FALSE_NEGATIVE
1,0,0,fairmont,MTurk,My experience as Fairmont Chicago Millennium P...,"[0, 0]",FALSE_NEGATIVE
2,0,0,omni,MTurk,When my boyfriend and I checked into our room ...,"[0, 0]",FALSE_NEGATIVE
3,0,0,hyatt,MTurk,My wife and I stayed at the Hyatt Regency in C...,"[0, 0]",FALSE_NEGATIVE
4,0,0,omni,MTurk,I was extremely disappointed with my stay at t...,"[0, 0]",FALSE_NEGATIVE
...,...,...,...,...,...,...,...
1595,1,0,sheraton,MTurk,What a gorgeous hotel! From the moment we step...,"[1, 0]",FALSE_POSITIVE
1596,1,0,homewood,MTurk,If you want the downtown experience of a lifet...,"[1, 0]",FALSE_POSITIVE
1597,1,0,knickerbocker,MTurk,I stayed at the Millennium Knickerbocker Hotel...,"[1, 0]",FALSE_POSITIVE
1598,1,0,homewood,MTurk,Downtown Chicago has a multitude of available ...,"[1, 0]",FALSE_POSITIVE


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

Y = data['given_class']
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)
dummy_y

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [ ]:
textData = pd.DataFrame(list(data['Review'])) # each row is one document; the raw text of the document should be in the 'Review' column

In [ ]:
import snowballstemmer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from string import ascii_lowercase

# initialize stemmer
stemmer = snowballstemmer.EnglishStemmer()

# grab stopword list, extend it a bit, and then turn it into a set for later
stop = stopwords.words('english')
stop.extend(['may','also','zero','one','two','three','four','five','six','seven','eight','nine','ten','across','among','beside','however','yet','within']+list(ascii_lowercase))
stoplist = stemmer.stemWords(stop)
stoplist = set(stoplist)
stop = set(sorted(stop + list(stoplist)))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import itertools
# remove characters and stoplist words, then generate dictionary of unique words
textData[0].replace('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ',inplace=True,regex=True)
wordlist = filter(None, " ".join(list(set(list(itertools.chain(*textData[0].str.split(' ')))))).split(" "))
data['stemmed_text_data'] = [' '.join(filter(None,filter(lambda word: word not in stop, line))) for line in textData[0].str.lower().str.split(' ')]
data

,Polarity,Deceptive,Hotel,Source,Review,final_class,given_class,stemmed_text_data
0,0,0,hyatt,MTurk,The Hyatt Regency Chicago basically caters to ...,"[0, 0]",FALSE_NEGATIVE,hyatt regency chicago basically caters guests ...
1,0,0,fairmont,MTurk,My experience as Fairmont Chicago Millennium P...,"[0, 0]",FALSE_NEGATIVE,experience fairmont chicago millennium park ho...
2,0,0,omni,MTurk,When my boyfriend and I checked into our room ...,"[0, 0]",FALSE_NEGATIVE,boyfriend checked room omni chicago irritated ...
3,0,0,hyatt,MTurk,My wife and I stayed at the Hyatt Regency in C...,"[0, 0]",FALSE_NEGATIVE,wife stayed hyatt regency chicago couple weeks...
4,0,0,omni,MTurk,I was extremely disappointed with my stay at t...,"[0, 0]",FALSE_NEGATIVE,extremely disappointed stay omni chicago hotel...
...,...,...,...,...,...,...,...,...
1595,1,0,sheraton,MTurk,What a gorgeous hotel! From the moment we step...,"[1, 0]",FALSE_POSITIVE,gorgeous hotel moment stepped inside warm colo...
1596,1,0,homewood,MTurk,If you want the downtown experience of a lifet...,"[1, 0]",FALSE_POSITIVE,want downtown experience lifetime historical l...
1597,1,0,knickerbocker,MTurk,I stayed at the Millennium Knickerbocker Hotel...,"[1, 0]",FALSE_POSITIVE,stayed millennium knickerbocker hotel total da...
1598,1,0,homewood,MTurk,Downtown Chicago has a multitude of available ...,"[1, 0]",FALSE_POSITIVE,downtown chicago multitude available hotels yo...


In [ ]:
from collections import Counter
import re
# remove all words that don't occur at least 5 times and then stem the resulting docs
minimum_count = 1
str_frequencies = pd.DataFrame(list(Counter(filter(None,list(itertools.chain(*data['stemmed_text_data'].str.split(' '))))).items()),columns=['word','count'])
low_frequency_words = set(str_frequencies[str_frequencies['count'] < minimum_count]['word'])
data['stemmed_text_data'] = [' '.join(filter(None,filter(lambda word: word not in low_frequency_words, line))) for line in data['stemmed_text_data'].str.split(' ')]
data['stemmed_text_data'] = [" ".join(stemmer.stemWords(re.sub('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ', next_text).split(' '))) for next_text in data['stemmed_text_data']]
data

,Polarity,Deceptive,Hotel,Source,Review,final_class,given_class,stemmed_text_data
0,0,0,hyatt,MTurk,The Hyatt Regency Chicago basically caters to ...,"[0, 0]",FALSE_NEGATIVE,hyatt regenc chicago basic cater guest want fe...
1,0,0,fairmont,MTurk,My experience as Fairmont Chicago Millennium P...,"[0, 0]",FALSE_NEGATIVE,experi fairmont chicago millennium park horrib...
2,0,0,omni,MTurk,When my boyfriend and I checked into our room ...,"[0, 0]",FALSE_NEGATIVE,boyfriend check room omni chicago irrit see be...
3,0,0,hyatt,MTurk,My wife and I stayed at the Hyatt Regency in C...,"[0, 0]",FALSE_NEGATIVE,wife stay hyatt regenc chicago coupl week ago ...
4,0,0,omni,MTurk,I was extremely disappointed with my stay at t...,"[0, 0]",FALSE_NEGATIVE,extrem disappoint stay omni chicago hotel hot ...
...,...,...,...,...,...,...,...,...
1595,1,0,sheraton,MTurk,What a gorgeous hotel! From the moment we step...,"[1, 0]",FALSE_POSITIVE,gorgeous hotel moment step insid warm color lo...
1596,1,0,homewood,MTurk,If you want the downtown experience of a lifet...,"[1, 0]",FALSE_POSITIVE,want downtown experi lifetim histor live bring...
1597,1,0,knickerbocker,MTurk,I stayed at the Millennium Knickerbocker Hotel...,"[1, 0]",FALSE_POSITIVE,stay millennium knickerbock hotel total day st...
1598,1,0,homewood,MTurk,Downtown Chicago has a multitude of available ...,"[1, 0]",FALSE_POSITIVE,downtown chicago multitud avail hotel youv dec...
